# This notebook

This notebook contains first bits and pieces of the yet to be developed model correlating climate/environmental factors with conflict occurrence.

## Import libraries and file with settings

In [1]:
import conflict_model

import pandas as pd
import geopandas as gpd
from configparser import RawConfigParser
import matplotlib.pyplot as plt
import numpy as np
import datetime
import csv
import netCDF4 as nc
import rasterstats as rstats
import xarray as xr
import rasterio as rio
import seaborn as sbs
from sklearn import svm, preprocessing, model_selection, metrics
from shutil import copyfile
import os, sys

In [2]:
conflict_model.utils.show_versions()

Python version: 3.7.7 (default, Apr 15 2020, 05:09:04) [MSC v.1916 64 bit (AMD64)]
conflict_model version: 0.0.2b1
geopandas version: 0.7.0
xarray version: 0.15.1
rasterio version: 1.1.0
pandas version: 1.0.3
numpy version: 1.18.1
scikit-learn version: 0.22.1
matplotlib version: 3.2.1
seaborn version: 0.10.1
rasterstats version: 0.14.0


Geopandas versions lower than 0.7.0 do not yet have the clip function. The notebook will thus not work with these versions.

In [3]:
if gpd.__version__ < '0.7.0':
    sys.exit('please upgrade geopandas to version 0.7.0, your current version is {}'.format(gpd.__version__))

In this file all the settings for the analysis are defined. By 'parsing' it, all values are read for different sections. This is a simple way to make the code independent of the input data and settings.

In [4]:
settings_file = r'../data/run_setting.cfg'

In [5]:
config = RawConfigParser(allow_no_value=True)
config.read(settings_file);

In [6]:
#out_dir
out_dir = config.get('general','output_dir')
if not os.path.isdir(out_dir):
        os.makedirs(out_dir)
print('for the record, saving output to folder {}'.format(out_dir) + os.linesep)

for the record, saving output to folder C:\Users\hoch0001\Documents\_code\conflict_model\data\OUT



In [7]:
copyfile(settings_file, os.path.join(out_dir, 'copy_of_run_setting.cfg'));

# Applying functions

In [8]:
gdf = conflict_model.utils.get_geodataframe(config)

reading csv file to dataframe C:\Users\hoch0001\Documents\_code\conflict_model\data\UCDP/ged191.csv
...DONE

translating to geopandas dataframe
...DONE



In [9]:
conflict_gdf, extent_gdf = conflict_model.selection.select(gdf, config)

filtering on conflict properties...
...filtering key best with lower value 5
...filtering key type_of_violence with value 1
...passing key country as it is empty
focussing on period between 2000 and 2011

reading extent and spatial aggregation level from file C:\Users\hoch0001\Documents\_code\conflict_model\data\waterProvinces/waterProvinces_Africa.shp
...DONE

clipping datasets to extent
...DONE

clipping to climate zones['BWh', 'BSh']
...DONE



# Functions

## Plotting

In [10]:
def make_meshgrid(x, y, h=.02):
    """Create a mesh of points to plot in

    Parameters
    ----------
    x: data to base x-axis meshgrid on
    y: data to base y-axis meshgrid on
    h: stepsize for meshgrid, optional

    Returns
    -------
    xx, yy : ndarray
    """
    x_min, x_max = x.min() - 1, x.max() + 1
    y_min, y_max = y.min() - 1, y.max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    return xx, yy

In [11]:
def plot_contours(ax, clf, xx, yy, **params):
    """Plot the decision boundaries for a classifier.

    Parameters
    ----------
    ax: matplotlib axes object
    clf: a classifier
    xx: meshgrid ndarray
    yy: meshgrid ndarray
    params: dictionary of params to pass to contourf, optional
    """
    Z = clf.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    out = ax.contourf(xx, yy, Z, **params)
    return out

# Analysis per year

In a first step, we want to know in which countries there was conflict or not. To that end, we first accumulate the number of fatalities per country and use this as proxy whether there was a conlfict or not (guess there is a rather strong like...).

In [12]:
print('simulation period from', str(config.getint('settings', 'y_start')), 'to', str(config.getint('settings', 'y_end')))
print('')

X1 = pd.Series(dtype=float)
X2 = pd.Series(dtype=float)
X3 = pd.Series(dtype=float)
Y  = pd.Series(dtype=int) # not bool, because otherwise 0 is converted to False and 1 to True but we need 0/1

# go through all simulation years as specified in config-file
for sim_year in np.arange(config.getint('settings', 'y_start'), config.getint('settings', 'y_end'), 1):
    
    print('entering year {}'.format(sim_year) + os.linesep)
    
    list_boolConflict = conflict_model.get_boolean_conflict.conflict_in_year_bool(conflict_gdf, extent_gdf, config, sim_year)
    Y = Y.append(pd.Series(list_boolConflict, dtype=int), ignore_index=True)
    
    list_GDP_PPP = conflict_model.get_var_from_nc.nc_with_integer_timestamp(extent_gdf, config, 'GDP_per_capita_PPP', sim_year)
    X1 = X1.append(pd.Series(list_GDP_PPP), ignore_index=True)
    
    if not len(list_GDP_PPP) == len(list_boolConflict):
        raise AssertionError('length of lists do not match, they are {0} and {1}'.format(len(list_GDP_PPP), len(list_boolConflict)))
    
    list_Evap = conflict_model.get_var_from_nc.nc_with_continous_regular_timestamp(extent_gdf, config, 'total_evaporation', sim_year)
    X2 = X2.append(pd.Series(list_Evap), ignore_index=True)
    
    if not len(list_Evap) == len(list_boolConflict):
        raise AssertionError('length of lists do not match, they are {0} and {1}'.format(len(list_Evap), len(list_boolConflict)))
        
    list_precip = conflict_model.get_var_from_nc.nc_with_continous_regular_timestamp(extent_gdf, config, 'precipitation', sim_year)
    X3 = X3.append(pd.Series(list_precip), ignore_index=True)   
    
print('...simulation DONE')

simulation period from 2000 to 2011

entering year 2000

determining whether a conflict took place or not
...DONE

calculating mean GDP_per_capita_PPP per aggregation unit from file C:\Users\hoch0001\Documents\_code\conflict_model\data\GDP_HDI/GDP_per_capita_PPP_1990_2015_Africa.nc for year 2000


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

calculating mean total_evaporation per aggregation unit from file C:\Users\hoch0001\Documents\_code\conflict_model\data\PCRGLOBWB/totalEvap/totalEvaporation_monthTot_output_2000_2015_Africa_yearmean.nc for year 2000


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

calculating mean precipitation per aggregation unit from file C:\Users\hoch0001\Documents\_code\conflict_model\data\PCRGLOBWB/precip/cruts321_erainterim_daily_precipitation_2000_to_2010_yearmean_Africa.nc for year 2000


...

...DONE

calculating mean total_evaporation per aggregation unit from file C:\Users\hoch0001\Documents\_code\conflict_model\data\PCRGLOBWB/totalEvap/totalEvaporation_monthTot_output_2000_2015_Africa_yearmean.nc for year 2010


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

calculating mean precipitation per aggregation unit from file C:\Users\hoch0001\Documents\_code\conflict_model\data\PCRGLOBWB/precip/cruts321_erainterim_daily_precipitation_2000_to_2010_yearmean_Africa.nc for year 2010


C:\Users\hoch0001\AppData\Local\Continuum\anaconda3\envs\conflict_model\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


...DONE

...simulation DONE


# Machine Learning

## Data preparation

First, create a pandas dataframe from all variables and targets and kick out rows with missing values (they do not work with ML)

In [13]:
XY_data = list(zip(X1, X2, X3, Y))
XY_data = pd.DataFrame(XY_data, columns=['GDP_PPP', 'ET', 'P', 'conflict'])
print('number of data points including missinv values:', len(XY_data))
XY_data = XY_data.dropna()
print('number of data points excluding missinv values:', len(XY_data))

number of data points including missinv values: 4246
number of data points excluding missinv values: 4224


In [14]:
XY_data

,GDP_PPP,ET,P,conflict
0,2361.934264,0.042316,0.001967,0
1,3104.051687,0.040520,0.001448,0
2,1192.025215,0.039277,0.001211,0
3,1275.859490,0.025305,0.000869,0
4,1182.202026,0.036308,0.001356,0
...,...,...,...,...
4241,3277.156738,0.060997,0.002308,0
4242,3277.156738,0.068696,0.002632,0
4243,1381.966901,0.048329,0.002686,0
4244,1390.211488,0.052157,0.002232,0


Then, convert them to numpy arrays

In [15]:
X = XY_data[['GDP_PPP', 'ET', 'P']].to_numpy()
X

array([[2.36193426e+03, 4.23162297e-02, 1.96707654e-03],
       [3.10405169e+03, 4.05202232e-02, 1.44826607e-03],
       [1.19202521e+03, 3.92765536e-02, 1.21141790e-03],
       ...,
       [1.38196690e+03, 4.83292063e-02, 2.68591560e-03],
       [1.39021149e+03, 5.21571179e-02, 2.23178339e-03],
       [1.39168983e+03, 5.28718745e-02, 1.94693414e-03]])

In [16]:
Y = XY_data.conflict.astype(int).to_numpy()
Y

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# X_plot = X[:, :2]

# clf = svm.SVC().fit(X_plot, Y)

# xx, yy = make_meshgrid(X_plot[:, 0], X_plot[:, 1])

# fig, ax = plt.subplots(1, 1, figsize=(20,10))
# plot_contours(ax, clf, xx, yy,
#               cmap=plt.cm.coolwarm, alpha=0.8)
# ax.scatter(X_plot[:, 0], X_plot[:, 1], c=y, cmap=plt.cm.coolwarm, s=20, edgecolors='k')
# ax.set_xlim(xx.min(), xx.max())
# ax.set_ylim(yy.min(), yy.max())
# ax.set_xlabel('Sepal length')
# ax.set_ylabel('Sepal width')
# ax.set_xticks(())
# ax.set_yticks(())
# ax.set_title('title')

### Target evaluation

In [ ]:
print('the total number of data points for our target is', len(Y))

In [ ]:
print('from this, {0} points are equal to 1, i.e. represent conflict occurence. This is a fraction of {1} percent.'.format(len(np.where(Y != 0)[0]), round(100*len(np.where(Y != 0)[0])/len(Y), 2)))

## Data preprocessing

Before we can train and predict with the model, we need to scale the variable data and create trainings and test data for both variables and target.

There are different scaling algorithms available. For our application, the MinMaxScaler and StandardScaler are the most obvious choices, the RobustScaler is follow-up. Depending on how incoming data looks like and is distributed, the scaling decision may need to be updated.

Depending on which scaler is chosen, the standardization of the data follows different approaches and may eventually influence model results. See here for some info: https://scikit-learn.org/stable/modules/preprocessing.html

In [ ]:
scaler = preprocessing.MinMaxScaler()
# scaler = preprocessing.StandardScaler()
# scaler = preprocessing.RobustScaler()

The scaler is then used to fit the data and transform it according to scaler-specific method.

In [ ]:
X_scaled = scaler.fit_transform(X)

The scaled variable data X_scaled is, together with the target data Y, split into trainings and test data. The fraction of the total data that is used for training is user-defined.

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_scaled,
                                                                    Y,
                                                                    test_size=1-config.getfloat('machine_learning', 'train_fraction'))

The scatterplot of the (two) scaled variables in X_trainwe looks like this. Also the sample size n_train is provided used to train the data alongside with the total variable sample size n_tot.

In [ ]:
plt.figure(figsize=(20,10))
sbs.scatterplot(x=X_train[:,0],
                y=X_train[:,1],  
                hue=y_train)

plt.title('training-data scaled with {0}; n_train={1}; n_tot={2}'.format(str(scaler).rsplit('(')[0], len(X_train), len(X_scaled)))
plt.xlabel('Variable 1')
plt.ylabel('Variable 2')
plt.savefig(os.path.join(out_dir, 'scatter_plot_scaled_traindata_{}.png'.format(str(scaler).rsplit('(')[0])), dpi=300)
plt.show()

A brief look at the statistics of the data of our scaled and standardized variables.

In [ ]:
X_scaled.mean(axis=0), X_scaled.std(axis=0)

## Model

### Train and predict

Create Support Vector Classification (SVC) model with balanced weight since data is unbalanced (e.g. many negative and few positive).

Note that there are many many settings in the svm.SVC class to be altered. At the moment, we stick to defaults besides the class_weight.

In [ ]:
clf = svm.SVC(class_weight='balanced')

Fit the model with the scaled training data and the boolean conflict data

In [ ]:
clf.fit(X_train, y_train);

Predict with the scaled prediction data. We see an array with lots of 0 and 1 is produced, similar as the target data of the model, y_test.

In [ ]:
y_pred = clf.predict(X_test)
y_pred

No clue right now what this does... look it up!

In [ ]:
y_score = clf.decision_function(X_test)
y_score

### Evaluation

The **accuracy** is either the fraction (default) or the count (normalize=False) of correct predictions.

The **precision** is the ratio *tp / (tp + fp)* where tp is the number of true positives and fp the number of false positives. The precision is intuitively the ability of the classifier not to label as positive a sample that is negative.

The **recall** is the ratio *tp / (tp + fn)* where tp is the number of true positives and fn the number of false negatives. The recall is intuitively the ability of the classifier to find all the positive samples.

In [ ]:
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))
print("Precision:", metrics.precision_score(y_test, y_pred))
print("Recall:", metrics.recall_score(y_test, y_pred))

**Precision-Recall** is a useful measure of success of prediction when the classes are very imbalanced. In information retrieval, precision is a measure of result relevancy, while recall is a measure of how many truly relevant results are returned.

The precision-recall curve shows the tradeoff between precision and recall for different threshold. A high area under the curve represents both high recall and high precision, where high precision relates to a low false positive rate, and high recall relates to a low false negative rate. High scores for both show that the classifier is returning accurate results (high precision), as well as returning a majority of all positive results (high recall).

A system with high recall but low precision returns many results, but most of its predicted labels are incorrect when compared to the training labels. A system with high precision but low recall is just the opposite, returning very few results, but most of its predicted labels are correct when compared to the training labels. An ideal system with high precision and high recall will return many results, with all results labeled correctly.

In [ ]:
average_precision = metrics.average_precision_score(y_test, y_score)

print('Average precision-recall score: {0:0.2f}'.format(average_precision))

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20,10))
disp = metrics.plot_precision_recall_curve(clf, X_test, y_test, ax=ax)
disp.ax_.set_title('2-class Precision-Recall curve: AP={} with {}'.format(round(average_precision,2), str(scaler).rsplit('(')[0]))
plt.savefig(os.path.join(out_dir, 'precision_recall_curve_{}.png'.format(str(scaler).rsplit('(')[0])), dpi=300)

Results are pretty crappy, but that is okay give we use not very sensible input data at the moment...

# Documentation

Let's safe some settings used in this run to csv-files so results can be assessed in light of these settings.

First, the parameters of our SVM model.

In [ ]:
SVC_params = clf.get_params()

out_fo = os.path.join(out_dir, 'SVC_params.csv')
w = csv.writer(open(out_fo, "w"))
for key, val in SVC_params.items():
    w.writerow([key, val])

Second, the parameters of the scaling method we used.

In [ ]:
scaler_params = scaler.get_params()

out_fo = os.path.join(out_dir, '{}_params.csv'.format(str(scaler).rsplit('(')[0]))
w = csv.writer(open(out_fo, "w"))
for key, val in scaler_params.items():
    w.writerow([key, val])

Third, the resulting values of our objective functions.

In [ ]:
evaluation = {'Accuracy': round(metrics.accuracy_score(y_test, y_pred), 2),
              'Precision': round(metrics.precision_score(y_test, y_pred), 2),
              'Recall': round(metrics.recall_score(y_test, y_pred), 2),
              'Average precision-recall score': round(average_precision, 2)}

out_fo = os.path.join(out_dir, 'evaluation.csv')
w = csv.writer(open(out_fo, "w"))
for key, val in evaluation.items():
    w.writerow([key, val])